In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam

In [2]:
  #*----------------------------------------------------------------------------*
#* Copyright (C) 2020 ETH Zurich, Switzerland                                 *
#* SPDX-License-Identifier: Apache-2.0                                        *
#*                                                                            *
#* Licensed under the Apache License, Version 2.0 (the "License");            *
#* you may not use this file except in compliance with the License.           *
#* You may obtain a copy of the License at                                    *
#*                                                                            *
#* http://www.apache.org/licenses/LICENSE-2.0                                 *
#*                                                                            *
#* Unless required by applicable law or agreed to in writing, software        *
#* distributed under the License is distributed on an "AS IS" BASIS,          *
#* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.   *
#* See the License for the specific language governing permissions and        *
#* limitations under the License.                                             *
#*                                                                            *
#* Authors: Batuhan Toemekce, Burak Kaya, Michael Hersche                     *
#*----------------------------------------------------------------------------*

#!/usr/bin/env python3

__author__ = "Michael Hersche"
__email__ = "herschmi@iis.ee.ethz.ch"

import numpy as np
import scipy.signal as scp
import pdb


def eeg_reduction(x, n_ds = 1, n_ch = 64, T = 3, fs = 160):
	'''
	Inputs
	------
	x : np array ()
		input array
	n_ds: int
		downsampling factor
	n_ch: int
		number of channels
	T: float
		time [s] to classify
	fs: int
		samlping frequency [Hz]


	Outputs
	-------
	'''


	if n_ch ==64:
		channels = np.arange(0,n_ch)
	elif n_ch == 38:
		channels = np.array([0,2,4,6,8,10,12,14,16,18,20,21,22,23,24,26,28,29,31,33,35,37,40,41,42,43,46,48,50,52,54,55,57,59,60,61,62,63])
	elif n_ch == 27:
		channels = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,38,39,40,41,44,45])
	elif n_ch == 19:
		channels = np.array([8,10,12,21,23,29,31,33,35,37,40,41,46,48,50,52,54,60,62])
	elif n_ch ==8:
		channels = np.array([8,10,12,25,27,48,52,57])


	n_s_orig = int(T*fs)
	n_s = int(np.ceil(T*fs/n_ds)) # number of time samples
	n_trial = x.shape[0]

	# channel selection
	if n_ds >1:
		x = x[:,channels]
		y = np.zeros((n_trial, n_ch,n_s))
		for trial in range(n_trial):
			for chan in range(n_ch):
				# downsampling
				#pdb.set_trace()
				y[trial,chan] = scp.decimate(x[trial,chan,:n_s_orig],n_ds)
	else:
		y = x[:,channels]
		y = y[:,:,:n_s_orig]

	return y

In [3]:
#*----------------------------------------------------------------------------*
#* Copyright (C) 2020 ETH Zurich, Switzerland                                 *
#* SPDX-License-Identifier: Apache-2.0                                        *
#*                                                                            *
#* Licensed under the Apache License, Version 2.0 (the "License");            *
#* you may not use this file except in compliance with the License.           *
#* You may obtain a copy of the License at                                    *
#*                                                                            *
#* http://www.apache.org/licenses/LICENSE-2.0                                 *
#*                                                                            *
#* Unless required by applicable law or agreed to in writing, software        *
#* distributed under the License is distributed on an "AS IS" BASIS,          *
#* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.   *
#* See the License for the specific language governing permissions and        *
#* limitations under the License.                                             *
#*                                                                            *
#* Authors: Batuhan Toemekce, Burak Kaya, Michael Hersche                     *
#*----------------------------------------------------------------------------*

#!/usr/bin/env python3

"""
Loads '.edf' MI data from Physionet
"""

import os
import numpy as np
# pyEDFlib is a python library to read/write EDF+/BDF+ files based on EDFlib.
import pyedflib as edf
import statistics as stats
import random


__author__ = "Batuhan Tomekce, Burak Alp Kaya, Michael Hersche"
__email__ = "tbatuhan@ethz.ch, bukaya@ethz.ch, herschmi@ethz.ch"

def get_data(path, long = False, normalization = 0,subjects_list=range(1,110), n_classes=4):
    '''
    Load data samples and return it as one array

    Parameters:
    -----------
    path:   string
        path to .edf data of
    normalization   int {0,1}
        normalization per trial
        0: no normalization; 1: normalized per channel
    long:    bool
        length of read time window
        True: Trials of length 6s returned; False: Trials of length 3s returned
    subjects_list   list [1, .. , 109]
        list of subject numbers to be loaded
    n_classes:      int
        number of classes
        2: L/R, 3: L/R/0, 4

    Return: X:  numpy array (n_sub*n_trials, 64, n_samples)
                EEG data
            y:  numpy array (n_sub*n_trials, n_samples)
                labels
    '''
    # Define subjects whose data is not taken, for details see data tester added 106 again to analyze it, deleted from the exluded list
    excluded_subjects = [88,92,100,104]
    # Define subjects whose data is taken, namely from 1 to 109 excluding excluded_subjects
    subjects = [x for x in subjects_list if (x not in excluded_subjects)]


    mi_runs = [1, 4, 6, 8, 10, 12, 14]
    # Extract only requested number of classes
    if(n_classes == 3):
        print('Returning 3 Class data')
        mi_runs.remove(6) # feet
        mi_runs.remove(10) # feet
        mi_runs.remove(14) # feet
    elif(n_classes == 2):
        print('Returning 2 Class data')
        mi_runs.remove(6) # feet
        mi_runs.remove(10) # feet
        mi_runs.remove(14) # feet
        mi_runs.remove(1) #rest
    print(f'Data from runs: {mi_runs}')

    X, y = read_data(subjects = subjects,runs = mi_runs, path=path, long=long)

    # do normalization if wanted
    if(normalization == 1):
        #TODO: declare std_dev, mean arrays to return
        for i in range(X.shape[0]):
            for ii in range(X.shape[1]):
                std_dev = stats.stdev(X[i,ii])
                mean = stats.mean(X[i,ii])
                X[i,ii] = (X[i,ii] - mean) / std_dev

    return X, y



def read_data(subjects , runs, path, long=False):
    '''
    Load data samples and return it as one array

    Parameters:
    -----------
    subjects   list [1, .. , 109]
        list of subject numbers to be loaded
    path:   string
        path to .edf data of
    runs    list
        runs to read from
    long:    bool
        length of read time window
        True: Trials of length 6s returned; False: Trials of length 3s returned


    Return: X:  numpy array (n_sub*n_trials, 64, n_samples)
                EEG data
            y:  numpy array (n_sub*n_trials, n_samples)
                labels
    '''

    """
    DATA EXPLANATION:

        LABELS:
        both first_set and second_set
            T0: rest
        first_set (real motion in runs 3, 7, and 11; imagined motion in runs 4, 8, and 12)
            T1: the left fist
            T2: the right fist
        second_set (real motion in runs 5, 9, and 13; imagined motion in runs 6, 10, and 14)
            T1: both fists
            T2: both feet

        Here, we get data from the first_set (rest, left fist, right fist),
        and also data from the second_set (rest, both feet).
        We ignore data for T1 from the second_set and thus return data for
        four classes/categories of events: Rest, Left Fist, Right Fist, Both Feet.
    """
    base_file_name = 'S{:03d}R{:02d}.edf'
    base_subject_directory = 'S{:03d}'

    # Define runs where the two different sets of tasks were performed
    baseline = np.array([1])
    first_set = np.array([4,8,12])
    second_set = np.array([6,10,14])

    # Number of EEG channels
    NO_channels = 64
    # Number of Trials extracted per Run
    NO_trials = 7

    # Define Sample size per Trial
    if not long:
        n_samples = int(160 * 3) # 3s Trials: 480 samples
    else:
        n_samples = int(160 * 6) # 6s Trials: 960 samples

    # initialize empty arrays to concatanate with itself later
    X = np.empty((0,NO_channels,n_samples))
    y = np.empty(0)

    for subject in subjects:

        for run in runs:
            #For each run, a certain number of trials from corresponding classes should be extracted
            counter_0 = 0
            counter_L = 0
            counter_R = 0
            counter_F = 0

            # Create file name variable to access edf file
            filename = base_file_name.format(subject,run)
            directory = base_subject_directory.format(subject)
            file_name = os.path.join(path,directory,filename)
            # Read file
            f = edf.EdfReader(file_name)
            # Signal Parameters - measurement frequency
            fs = f.getSampleFrequency(0)
            # Number of eeg channels = number of signals in file
            n_ch = f.signals_in_file
            # Initiate eg.: 64*20000 matrix to hold all datapoints
            sigbufs = np.zeros((n_ch, f.getNSamples()[0]))

            for ch in np.arange(n_ch):
                # Fill the matrix with all datapoints from each channel
                sigbufs[ch, :] = f.readSignal(ch)

            # Get Label information
            annotations = f.readAnnotations()

            # close the file
            f.close()

            # Get the specific label information
            labels = annotations[2]
            points = fs*annotations[0]

            labels_int = np.empty(0)
            data_step = np.empty((0,NO_channels, n_samples))

            if run in second_set:
                for ii in range(0,np.size(labels)):
                    if(labels[ii] == 'T0' and counter_0 < NO_trials):
                        continue
                        counter_0 += 1
                        labels_int = np.append(labels_int,[2])

                    elif(labels[ii] == 'T2' and counter_F < NO_trials):
                        counter_F += 1
                        labels_int = np.append(labels_int,[3])
                        # change data shape and seperate events
                        data_step = np.vstack((data_step, np.array(sigbufs[:,int(points[ii]):int(points[ii])+n_samples])[None]))

            elif run in first_set:
                for ii in range(0,np.size(labels)):
                    if(labels[ii] == 'T0' and counter_0 < NO_trials):
                        continue
                        counter_0 += 1
                        labels_int = np.append(labels_int, [2])

                    elif(labels[ii] == 'T1' and counter_L < NO_trials):
                        counter_L += 1
                        labels_int = np.append(labels_int, [0])
                        data_step = np.vstack((data_step, np.array(sigbufs[:,int(points[ii]):int(points[ii])+n_samples])[None]))

                    elif(labels[ii] == 'T2' and counter_R < NO_trials):
                        counter_R += 1
                        labels_int = np.append(labels_int, [1])
                        data_step = np.vstack((data_step, np.array(sigbufs[:,int(points[ii]):int(points[ii])+n_samples])[None]))

            elif run in baseline:
                for ii in range(0,20):
                    if(counter_0 < 20):
                        counter_0 += 1
                        labels_int = np.append(labels_int, [2])
                        data_step = np.vstack((data_step, np.array(sigbufs[:,(ii*n_samples):((ii+1)*n_samples)])[None]))
                # randomly choose resting trials
                np.random.seed(7)
                index = random.randint(0*fs,57*fs)
                labels_int = np.append(labels_int, [2])
                data_step = np.vstack((data_step, np.array(sigbufs[:,(index):(index+n_samples)])[None]))

            # concatenate arrays in order to get the whole data in one input array
            X = np.concatenate((X,data_step))
            y = np.concatenate((y,labels_int))

    return X, y

In [4]:
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, ReLU, BatchNormalization, add,Softmax, AveragePooling2D, Dense, Input, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import SpatialDropout2D, Dropout
from tensorflow.keras.optimizers import Adam


def expansion_block(x,t,filters,block_id):
    prefix = 'block_{}_'.format(block_id)
    total_filters = t*filters
    x = Conv2D(total_filters,1,padding='same',use_bias=False, name =    prefix +'expand')(x)
    x = BatchNormalization(name=prefix +'expand_bn')(x)
    x = ReLU(6,name = prefix +'expand_relu')(x)
    x = Dropout(0.2)(x)
    return x

def depthwise_block(x,stride,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = DepthwiseConv2D(3,strides=(stride,stride),padding ='same', use_bias = False, name = prefix + 'depthwise_conv')(x)
    x = BatchNormalization(name=prefix +'dw_bn')(x)
    x = ReLU(6,name = prefix +'dw_relu')(x)
    x = Dropout(0.2)(x)
    return x

def projection_block(x,out_channels,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = Conv2D(filters=out_channels,kernel_size = 1,   padding='same',use_bias=False,name= prefix + 'compress')(x)
    x = BatchNormalization(name=prefix +'compress_bn')(x)
    x = ReLU(6,name = prefix +'projection_relu')(x) 
    x = Dropout(0.2)(x)
    return x

def Bottleneck(x,t,filters, out_channels,stride,block_id):
    y = expansion_block(x,t,filters,block_id)
    y = depthwise_block(y,stride,block_id)
    y = projection_block(y, out_channels,block_id)
    if y.shape[-1]==x.shape[-1]:
        y = add([x,y])
    return y

def MobileNetV2(nb_classes = 4, Chans = 64, Samples = 0):

    # if dropoutType == 'SpatialDropout2D':
    #     dropoutType = SpatialDropout2D
    # elif dropoutType == 'Dropout':
    #     dropoutType = Dropout
    input = Input((Chans, Samples, 1))

    x = Conv2D(32,kernel_size=3,strides=(2,2),padding = 'same', use_bias=False)(input)
    x = BatchNormalization(name='conv1_bn')(x)
    x = ReLU(6, name = 'conv1_relu')(x)

    # 17 Bottlenecks

    x = depthwise_block(x,stride=1,block_id=1)
    x = projection_block(x, out_channels=16,block_id=1)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 24, stride = 2,block_id = 2)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 24, stride = 1,block_id = 3)

    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 2,block_id = 4)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 5)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 6)

    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 2,block_id = 7)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 8)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 9)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 10)

    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 11)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 12)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 13)

    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 2,block_id = 14)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 15)
    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 16)

    # x = self.Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 320, stride = 1,block_id = 17)


    #1*1 conv
    #x = Conv2D(filters = 1280,kernel_size = 1,padding='same',use_bias=False, name = 'last_conv')(x)
    x = BatchNormalization(name='last_bn')(x)
    x = ReLU(6,name='last_relu')(x)

    #AvgPool 7*7
    x = GlobalAveragePooling2D(name='global_average_pool')(x)

    output = Dense(nb_classes,activation='softmax')(x)

    model = Model(input, output)

    return model

In [5]:
# Hyperparameters

num_classes_list = [4]
n_epochs = 2
n_ds = 1
n_ch = [64]
T = [3]

In [6]:
# Data Loading

# from google.colab import drive
# drive.mount('/content/gdrive')

# setting up directories
dir = '/content/gdrive/My Drive/egg_data/'

# Data Extraction from .edf format
# X, y = get(dir, n_classes=[4])
# np.savez('data',X_Train = X, y_Train = y)

# Load data
npzfile = np.load('data/data.npz')
X, y = npzfile['X_Train'], npzfile['y_Train']

In [7]:
# Data Reduction
X = eeg_reduction(X,n_ds = n_ds, n_ch = n_ch[0], T = T[0])
X = (np.expand_dims(X, axis=-1))
X.shape

(8820, 64, 480, 1)

In [8]:
def save_results(history):

    # Save metrics
    results = np.zeros((4,len(history['acc'])))
    results[0] = history['acc']
    results[1] = history['val_acc']
    results[2] = history['loss']
    results[3] = history['val_loss']
    return results[0:2,-1]

In [9]:
# parameter setting

n_samples = np.shape(X)[2]

# Training
results_dir = 'results/models/session1'
kf = KFold(n_splits = 4)
split_ctr = 0
y_cat = np_utils.to_categorical(y)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
data = kf.split(X, y)
for train, test in kf.split(X, y):
    # Model
    model = MobileNetV2(nb_classes = 4, Chans=64, Samples = np.shape(X)[2])
    # Set Learning Rate
    np.random.seed(42*(split_ctr+1))
    np.random.shuffle(train)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics = ['accuracy'])

    # Fitting the model

    history = model.fit(X[train], y_cat[train], validation_data=(X[test], y_cat[test]), epochs = 200, batch_size = 4)
    model.save(os.path.join(results_dir,f'{split_ctr}.keras'))
    split_ctr = split_ctr + 1

Epoch 1/200
1654/1654 [==============================] - 28s 15ms/step - loss: 1.3873 - accuracy: 0.2730 - val_loss: 1.4056 - val_accuracy: 0.2689
Epoch 2/200
1654/1654 [==============================] - 23s 14ms/step - loss: 1.3686 - accuracy: 0.3020 - val_loss: 1.4281 - val_accuracy: 0.3134
Epoch 3/200
1654/1654 [==============================] - 24s 14ms/step - loss: 1.3610 - accuracy: 0.3179 - val_loss: 1.4933 - val_accuracy: 0.3365
Epoch 4/200
1654/1654 [==============================] - 23s 14ms/step - loss: 1.3494 - accuracy: 0.3308 - val_loss: 1.4222 - val_accuracy: 0.3528
Epoch 5/200
1654/1654 [==============================] - 23s 14ms/step - loss: 1.3401 - accuracy: 0.3389 - val_loss: 1.3612 - val_accuracy: 0.3397
Epoch 6/200
1654/1654 [==============================] - 24s 14ms/step - loss: 1.3381 - accuracy: 0.3324 - val_loss: 1.5970 - val_accuracy: 0.3193
Epoch 7/200
1654/1654 [==============================] - 23s 14ms/step - loss: 1.3284 - accuracy: 0.3566 - val_loss: 1

KeyboardInterrupt: 